## 데이터셋 생성

### 1. 데이터셋 수집
### 2. 임베딩
### 3. Train, Test 나누기
### 4. 저장

In [1]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
from sklearn.model_selection import train_test_split
from jamo import h2j, j2hcj
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from keras.utils.np_utils import to_categorical
from glob import glob

c:\Users\choi seung un\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\choi seung un\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.el2c6ple4zyw3eceviv3oxxgrn2nrfm2.gfortran-win_amd64.dll
c:\Users\choi seung un\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
c:\Users\choi seung un\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
c:\Users\choi seung un\AppData\Local\Programs\Python\Python38\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
c:\Users\choi seung un\AppData\Local\Programs\Py

In [12]:
### 데이터셋 수집


nouns = list() # 이름 외의 데이터. label 0
names = list() # 이름 데이터. label 1

# name 데이터
for path in glob("../../data/name/*.txt"):
    with open(path, "rt", encoding="utf-8-sig") as f:
        while True:
            text = f.readline()[:-1]
            # 3글자인 텍스트만 가져온다
            if len(text) == 3: names.append(text)
            if not text:
                break

# etc 데이터 
for path in glob("../../data/etc/*.txt"):
    with open(path, "rt", encoding="utf-8-sig") as f:
        while True:
            text = f.readline()[:-1]
            # 3글자인 텍스트만 가져온다
            if len(text) == 3: nouns.append(text)
            if not text:
                break
            
nouns = set(nouns)
names = set(names)
nouns = list(nouns - names)
names = list(names)

print("이름이 아닌 단어 갯수: ", len(nouns), "\n",
      "이름 갯수: ", len(names))


df_0 = pd.DataFrame()
df_0["text"] = nouns
df_0["label"] = 0

df_1 = pd.DataFrame()
df_1["text"] = names
df_1["label"] = 1

df = pd.concat([df_0, df_1], axis=0)
df.reset_index(inplace=True, drop=True)
df

이름이 아닌 단어 갯수:  34566 
 이름 갯수:  47041


,text,label
0,냅스터,0
1,오리콘,0
2,매음도,0
3,금유리,0
4,우타방,0
...,...,...
81602,김온유,1
81603,임희남,1
81604,임성언,1
81605,고은비,1


In [13]:
## 임베딩

train_data = df["text"].progress_apply(lambda x: [ord(i) for i in j2hcj(h2j(x))])
train_data = np.array(train_data)
train_data = pad_sequences(train_data, maxlen=10, padding="post")
train_data

100%|██████████| 81607/81607 [00:01<00:00, 50095.24it/s]


array([[12596, 12624, 12610, ...,     0,     0,     0],
       [12615, 12631, 12601, ...,     0,     0,     0],
       [12609, 12624, 12615, ...,     0,     0,     0],
       ...,
       [12615, 12643, 12609, ..., 12627, 12596,     0],
       [12593, 12631, 12615, ...,     0,     0,     0],
       [12593, 12631, 12610, ...,     0,     0,     0]])

In [14]:
# Train, Test 나누기

random_seed = 1004
x = train_data
y = df["label"]

x_train, x_test, y_train, y_test = train_test_split(train_data, y, 
                                                    test_size=0.2, 
                                                    stratify=y, 
                                                    random_state=random_seed)

In [15]:
##### 라벨을 원핫인코딩
y_train = to_categorical(np.array(y_train))
y_test = to_categorical(np.array(y_test))

In [16]:
# 저장

path = "../../data/use_data/train_data.csv"
df.to_csv(path, encoding="utf-8-sig", index=False)

train_x_path = "../../data/use_data/train_x.npy"
test_x_path = "../../data/use_data/test_x.npy"
train_y_path = "../../data/use_data/train_y.npy"
test_y_path = "../../data/use_data/test_y.npy"

np.save(train_x_path, x_train)
np.save(test_x_path, x_test)
np.save(train_y_path, y_train)
np.save(test_y_path, y_test)


